## Dogs of the Dow Theory 「狗股」


> 狗股是投資美國股市的一個選股策略。原始狗股在道瓊斯工業指數內的股票里選取高股息率股票組合，現有其他變種，如小狗股等。也有人用類似的選股方式用在港股。
>
>　　對一心“執平貨”的投資者來說，股息率（dividend yields）是一個常用的指標；簡而言之，股息率愈高愈“抵買”，反之亦然。根據這個理念發展出來的“道指狗股理論”（Dogs of the Dow Theory），為一些懶於（或不懂）分析公司基本因素、從中尋找“超值”股票，卻又希望跑贏大市的散戶，提供了一個方便實用的選擇。
>
>　　狗股理論是美國基金經理邁克爾·奧希金斯（Michael O'Higgins）於1991年提出的跑贏大市投資策略。具體的做法是，投資者每年年底從道瓊斯工業平均指數成份股中找出10只股息率最高的股票，新年伊始買入，一年後再找出10只股息率最高的成分股，賣出手中不在名單中的股票，買入新上榜單的股票，每年年初年底都重覆這一投資動作，便可獲取超過大盤的回報。
>
>　　據有關統計，1975至1999年運用"狗股理論"，投資的平均複利回報達18%，遠高於市場3%的平均水平。但不幸的是，2008年，在百年一遇的金融危機中，10只“狗股”平均跌幅超過41％，跑輸了道指（34％）。不過，就像百年老店雷曼兄弟倒了一樣，2008年利用“狗股理論”的投資者輸得這麼慘，應該也屬於小概率事件。
>
>From [MBA 智库百科](https://wiki.mbalib.com/zh-tw/%E7%8B%97%E8%82%A1%E7%90%86%E8%AE%BA)

这里，我们通过两个量化交易库来帮助我们进行选股和回测 tushare & abupy, 当然，以后可能会用到 JQData

In [2]:
# 基础库导入

from __future__ import print_function
from __future__ import division

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tushare as ts
import datetime

import os
import sys
# 使用insert 0即只使用github，避免交叉使用了pip安装的abupy，导致的版本不一致问题
sys.path.insert(0, os.path.abspath('../'))
import abupy

# 使用沙盒数据，目的是和书中一样的数据环境
# abupy.env.enable_example_env_ipython()

from abupy import AbuFactorSellNDay, AbuFactorBuyWD, AbuPickStockNTop
from abupy import AbuFactorBuyBreak, AbuFactorAtrNStop, AbuFactorPreAtrNStop, AbuWeekMonthBuy
from abupy import abu, AbuFactorCloseAtrNStop, ABuProgress, AbuMetricsBase, EMarketTargetType
from abupy import AbuSymbolCN, ABuSymbolPd
from abupy import EMarketDataFetchMode, EMarketSourceType, EDataCacheType

NumExpr defaulting to 8 threads.


In [3]:
## Set up the date
today = '20' + datetime.date.today().strftime('%y%m%d')

index_all = ts.get_today_all()

df1 = index_all.copy()  #建立一个备份
n = 200  #选择前n个数据

df1['a']=[('ST' in x )for x in df1.name.astype(str)]  #先给ST股票做标记a
df1=df1.set_index('a')  #将a设置为索引 
df1=df1.drop(index=[True]) #删除ST股票
df1=df1.reset_index(drop=True) #重建默认索引

#删除业绩亏损的股票
df1 = df1[df1.per >0]

#删除净资产为负的股票
df1 = df1[df1.pb >0]

#选取市盈率前 200 名股票
df2 = df1.sort_values(by=['per'],ascending=True).head(n)
#print(len(df2))
#选取市净率 200 名股票
df3 = df1.sort_values(by=['pb'],ascending=True).head(n)
#print(len(df3))
#生成股票代码集合，进行集合运算
g2 = set(df2.code) #低市盈率股票代码
#print(len(g2))
g3 = set(df3.code) #低市净率股票代码
#print(len(g3))
g = g2&g3 #集合交运算
#print(len(g))
pending_list = list(g)  #把集合转为列表
print('Basic Selection:',pending_list, "Totally:", len(pending_list))

pending_list = pd.DataFrame({"Code":pending_list})

[Getting data:]############################################################Basic Selection: ['000825', '000709', '601000', '000040', '601005', '600308', '601169', '600859', '600269', '000683', '601186', '000039', '000415', '601077', '000778', '600694', '002133', '600716', '601390', '600188', '600348', '600012', '600282', '002092', '601398', '601009', '601838', '600926', '600000', '601288', '600623', '601229', '600051', '600382', '600782', '601166', '000926', '601998', '600153', '600350', '601939', '600638', '601669', '000959', '600997', '600823', '000898', '603518', '600016', '000980', '002936', '300089', '600971', '600019', '600708', '601818', '601997', '601988', '600808', '600919', '600067', '603323', '601577', '601328', '600743', '600376', '002619', '601699', '000791', '600657', '000069', '601800', '600123', '600569', '600015', '000402', '600740', '601828', '600231', '600665', '000900', '601588', '000732', '000620', '601101'] Totally: 85


In [4]:
df1.head()

,code,name,changepercent,trade,open,high,low,settlement,volume,turnoverratio,amount,per,pb,mktcap,nmc
0,688399,硕世生物,5.310,77.35,75.00,79.75,75.0,73.45,1.5216e+06,11.4078,1.1789e+08,53.345,9.095,4.5343e+05,103173.0654
1,688398,赛特新材,2.845,49.16,48.70,50.00,48.5,47.80,1.1803e+06,6.4846,5.8127e+07,68.278,9.518,3.9328e+05,89476.8042
2,688396,华润微,-2.523,40.95,42.71,42.78,40.2,42.01,2.4965e+07,10.5276,1.0275e+09,83.811,62.653,4.7992e+06,971075.9659
3,688389,普门科技,1.979,19.07,19.02,19.27,18.6,18.70,2.0159e+06,5.8007,3.8315e+07,90.810,10.713,8.0514e+05,66271.6921
4,688388,嘉元科技,-2.299,54.83,56.88,57.80,53.5,56.12,2.8448e+06,5.1094,1.5716e+08,53.755,5.141,1.2659e+06,305276.2124


In [ ]:
# 使用腾讯财经的数据 （比百度好）
abupy.env.g_market_source = EMarketSourceType.E_MARKET_SOURCE_tx
abupy.env.g_data_cache_type = EDataCacheType.E_DATA_CACHE_HDF5
# ABuSymbolPd.make_kl_df('601988')

# 设定时间区间和市场 n_jobs 就是并行数
abu.run_kl_update(start='2019-01-01', end='2020-01-01', market=EMarketTargetType.E_MARKET_TARGET_CN, n_jobs=10)

当然，我们应该把周期缩短一点，比如一个月一个月的筛选（毕竟政策市）

In [1]:
cash = 10000

# 延用周期突破策略做为买入因子
buy_factors = [{'xd': 21, 'class': AbuFactorBuyBreak},
               {'xd': 42, 'class': AbuFactorBuyBreak}]

# 卖出策略也还是继续延用
sell_factors = [
    {'stop_loss_n': 1.0, 'stop_win_n': 3.0,
     'class': AbuFactorAtrNStop},
    {'class': AbuFactorPreAtrNStop, 'pre_atr_n': 1.5},
    {'class': AbuFactorCloseAtrNStop, 'close_atr_n': 1.5}
]

def run_loo_back(choice_symbols, ps=None, n_folds=2, start=None, end=None, only_info=False):
    """封装一个回测函数"""
    if choice_symbols[0].startswith('us'):
        abupy.env.g_market_target = EMarketTargetType.E_MARKET_TARGET_US
    else:
        abupy.env.g_market_target = EMarketTargetType.E_MARKET_TARGET_CN
    abu_result_tuple, _ = abu.run_loop_back(cash,
                                           buy_factors,
                                           sell_factors,
                                           ps,
                                           start=start,
                                           end=end,
                                           n_folds=n_folds,
                                           choice_symbols=choice_symbols)
    ABuProgress.clear_output()
    AbuMetricsBase.show_general(*abu_result_tuple, returns_cmp=only_info, 
                                only_info=only_info,
                                only_show_returns=True)
    return abu_result_tuple

NameError: name 'AbuFactorBuyBreak' is not defined

In [ ]:
cn_choice_symbols = list(pending_list['Code'])
_ = run_loo_back(cn_choice_symbols)

# 我吐了 
$\vdots$

狗股理论使用的是参考值为股息率，很多基于狗股理论的选股策略进行了基因变种，如使用PEG替换股息率进行选股，或者直接使用上一年度的涨幅值做为选股参数，基于基本面数据进行选股的示例将在之后的章节进行示例，本节首先基于涨幅值做为狗股选股的参数。

abupy 中内置的选股因子AbuPickStockNTop即是在选股周期上对多只股票涨跌幅进行排序，选取top n 个股票做为交易目标，如下示例使用`AbuPickStockNTop` 在选股周期内选择涨幅最大的top3做为交易目标，如下：

In [ ]:
from abupy import AbuPickStockWorker, AbuPickRegressAngMinMax
from abupy import AbuBenchmark, AbuCapital, AbuKLManager, ABuRegUtil,ABuPickStockExecute

# 从kl_pd_manger缓存中获取选股走势数据，
# 注意get_pick_stock_kl_pd()为选股数据，get_pick_time_kl_pd()为择时
kl_pd_noah = kl_pd_manger.get_pick_stock_kl_pd('601988')
# 绘制并计算角度
deg = ABuRegUtil.calc_regress_deg(kl_pd_noah.close)
print('601988 选股周期内角度={}'.format(round(deg, 3)))

In [ ]:
# 选股条件threshold_ang_min=0.0, 即要求股票走势为向上上升趋势
stock_pickers = [{'class': AbuPickRegressAngMinMax,
                  'threshold_ang_min': 0.0, 'reversed': False}]

# 从这几个股票里进行选股，只是为了演示方便
# 一般的选股都会是数量比较多的情况比如全市场股票

benchmark = AbuBenchmark()
capital = AbuCapital(1000000, benchmark)
kl_pd_manger = AbuKLManager(benchmark, capital)
stock_pick = AbuPickStockWorker(capital, benchmark, kl_pd_manger,
                                choice_symbols=choice_symbols,
                                stock_pickers=stock_pickers)
stock_pick.fit()
# 打印最后的选股结果
stock_pick.choice_symbols